<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_LSTM%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%83%9D%EC%84%B1AI_%EB%AA%A8%EB%8D%B8%EB%A7%81_pytorch_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy

In [1]:
!pip install konlpy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

In [4]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 데이터 불러오기

### 전체 데이터셋

In [5]:
yo_review =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['고객리뷰']
yo_reply = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['사장댓글']
yo_star = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['별점']

train = pd.concat([yo_review, yo_reply, yo_star], axis=1)
rename_cols = ['review', 'reply', 'star']
train.columns = rename_cols
train['star'].value_counts().sort_index() # 별점 분포 확인

# shuffled_df = train.sample(frac=1, random_state=2)  # frac=1은 모든 행을 선택, random_state는 재현 가능한 랜덤 설정
# train = shuffled_df.iloc[:1000].reset_index(drop=True)

1       92
2       64
3      179
4      560
5    17791
Name: star, dtype: int64

In [6]:
print((92+64+179+560))
(92+64+179+560)/17791

895


0.05030633466359395

In [ ]:
# # 중복된 답변을 한 경우 행 전체 삭제
# train.drop_duplicates(subset='reply', keep=False, inplace=True)

### 오버샘플링(paraphrase 데이터)

In [7]:
train_over = pd.read_csv(f"{DATA_PATH}train_over.csv")
train_over

,review,reply,star
0,아니 그런데 배달이 20분 넘게 안 와서 전화해서 물었더니 배달 기사님이 비가 와서...,저희가 미처 생각하지 못한 부분이 있기 때문에 다시 한번 감사의 마음을 전하고 싶습니다.,1
1,양이 2만원씩이나하는데 장난으로하는것도 아니고 진짜 이거 2만원주고 사서 먹을 정도...,저희 1988응답하라추억의옛날도시락은 언제나 약속 드린 정량을 지키기 위해 노력하고...,1
2,커피가 배달되기를 기다린 지 25분이 지나자 커피 배달 예정 시간은 이미 커피를 만...,한파로 길이 꽁꽁 얼어 고객님들이 불편을 드려 죄송했는데 주문해 주셔서 감사합니다.,1
3,김이 많아서 그런것도 있고 단무짠지 김이 많이 없어서 그런지도 모르겠지만 밥이 쉬었...,이런 댓글은 본인 가족이나 지인에게 하는데 본인이 아니더라도 주변 사람이 해야 할 ...,1
4,처음 먹었을 때 너무 맛있게 먹었던 기억에 다시 찾아 주문하려고 잠실점에 갔는데 역...,잠실점에서 기분이 상하셨었는데 저희가 다시 실수를 한 것 같은데 다시 한번 연락드리...,1
...,...,...,...
721,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4
722,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4
723,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4
724,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4


In [8]:
train = pd.concat([train, train_over], axis=0).reset_index(drop=True)
train

,review,reply,star
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ...",5
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...,5
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...,5
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...,5
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...,5
...,...,...,...
19407,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4
19408,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4
19409,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4
19410,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4


## 데이터 정제

- 불필요한 문자 삭제

In [9]:
review_list = []
for review in train['review']:
    if isinstance(review, str):
        pat = re.compile("[^a-zA-Z가-힣0-9]")
        # pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

,review,reply,star
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ...",5
1,리뷰가 늦었네요 맛있게 잘 먹었습니다 또 시킬게요,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...,5
2,잘 먹었습니다 다만 주문에 있던 펩시콜라는 받지 못했습니다,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...,5
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...,5
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...,5


In [10]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):
        pat = re.compile("[^a-zA-Z가-힣0-9]")
        # pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

,review,reply,star
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요,이렇게 저희 매장 찾아주셔서 감사드립니다 맛 뿐만 아니라 양도 서비스도 모두 최상의...,5
1,리뷰가 늦었네요 맛있게 잘 먹었습니다 또 시킬게요,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,5
2,잘 먹었습니다 다만 주문에 있던 펩시콜라는 받지 못했습니다,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다 저희를 찾아 주신 만큼 1...,5
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다 저희 매장의 메뉴를 제대로 즐기신 거 ...,5
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,5


### "~~님" → "고객님"으로 변환

In [11]:
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)

### 답변 맞춤법 처리

In [ ]:
# !git clone https://github.com/ssut/py-hanspell.git
# !cd py-hanspell
# !pip install py-hanspell/

In [ ]:
# from hanspell import spell_checker
# def correct_spell(text):
#     if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
#         corrected_text = spell_checker.check(text).checked
#         return corrected_text
#     else:
#         return text

# correct_spell(train['reply'][0])

In [ ]:
# # 한시간 반 걸림. 원한다면 click
# train['reply'] = train['reply'].apply(correct_spell)

# 여기까지 완료된 데이터 불러오기

In [ ]:
# # # 전체
# # train = pd.read_csv(f"{DATA_PATH}yogiyo_clean.csv")

# # # 오버샘플
# # train = pd.read_csv(f"{DATA_PATH}yogiyo_OV_clean.csv")

# # paraphrase(가게명 처리만 안된)
# train = pd.read_csv(f"{DATA_PATH}yogiyo_paraphrased_reply.csv")
# train = train.drop(columns=["star","store"])
# train.head()

,review,reply
0,리뷰가 늦었네요 맛있게 잘 먹었습니다 또 시킬게요,힘든 주방일에 치이고 있는 와중에도 맛있는 요리 해 주시는 고객님 덕분에 저도 기분...
1,잘 먹었습니다 다만 주문에 있던 펩시콜라는 받지 못했습니다,저희 매장을 찾아와 주신 여러분께 감사의 말씀을 전하며 더 좋은 요리와 멋진 요리로...
2,굿 말이 필요 없는 메뉴 였습니다,저희 가게 앞에 와주신 많은 분들께 감사의 말씀 전합니다.
3,맛있어요 양도많구좋음,항상 노력하고 발전하는 곳에서 고객님의 만족도를 높여 드릴 것을 약속하겠습니다.
4,양많고 맛있었어요,저희 매장을 찾아와 주신 고객님들께 감사의 뜻을 전하고 싶고 또 그 고마움을 나눌 ...


### 가게명 키워드 컬럼화

In [12]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]
store_name = []
region_name = []
for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    if len(pat) >= 2:
        store_name.append(pat[0])
        region_name.append(pat[1])
    else:
        store_name.append(pat[0])

short_name = []
for name in store_name:
    for length in [2, 3, 4]:
        cut = name[:length]
        short_name.append(cut)
    if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
        cut = name[:6]
        short_name.append(cut)

short_name = [name for name in short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
short_name = list(set(short_name))
print(len(short_name), short_name)

# short_name 에서 불용어 처리
drop_name = ['OK','갈비','감성','걸작','고기','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교','손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '우시', '울진', '이끌', '이끌림', '인생', '직화', '진지', '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',]
short_name = [col for col in short_name if col not in drop_name]
print(len(short_name),short_name)

train['store'] = ""
for name in short_name:
    train[name] = train['reply'].apply(lambda x: name if name in x else '') # 각 가게 별로 컬럼 추가(그렇지 않으면 덮어쓰기 됨)
    train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # store 컬럼에 컬럼 값 합치기

train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 첫 번째 문자가 콤마인 경우 제거
train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(' ') else x) # 공백이어도 제거
train = train.drop(columns=short_name) # 각 가게 별 컬럼 삭제
train.head()

329 ['직화', '청년치킨', '진지덮', '대찌', '더팔당매', '송파만', '김포불', '창타이', '카페봄', '모모타코', '헬로팬닭', '초밥대', '윤희횟집', '고돼', '감성커', '우시장', '수상', '감성커피', '전금', '걸작떡', '최가네', '라화방', '부대찌', '최고', '수피자', '플러스8', '부성', '홍대개', '인생제육', '신자매', '모모타', '황궁쟁', '갈비', '시민', '왕빈자', '최고남', 'GTS', '청춘', '승도리', '부산오빠', '잭아저씨', '꼬알라', '킹프레소', '송파', '공주닭', '고기', '천년', '배달의쌀', '손수', '삼다', '명작', '대한', '39도시락', 'OK반', '큰아들백', '수제죽전', '창타이누', '더팔당', '시민보', '39도시', '텐텐마라', '병천', '무국적식', '병천토속', '땅끝', '최가네한', '갈비민족', '보돌미', '최고남제', '땅끝수', '39도', '구래상', '승도', '야키토', '킹프레', '부성초', 'GT', '텐텐', '더티', '폴트', '잭아', '저팔계', '텐텐마', '삽교', '호랑이초', '춘리마라', '청춘식당', '당치', '뜸들이다', '울진죽', '짱닭', '피자스', '올데이파', '신자매김', '깨돌이', '윤희횟', '폴트버거', '큰아들', '마라', '땅끝수산', '수락', '뜸들', '한식', '토핑', '닭강점', '비오키', '과일에반', '홍스', '과일에', '비오', '쇼부', '땡초곱', '큰아', '짱닭치', '피자보', '수상한삼', '한식세끼', '고기혁', '부산오', '직화삼겹', '짜글', '부대', '스시', '모어댄버', '고돼지', '플러스', '땡초곱창', '고씨네', '미친피자', '1989마라', '초밥', '이태', '행복한찜', '스시사', '시민보쌈', '여왕의', '황궁', '더티베지', '우리집', '피자보이', '조형훈족', '우시장

,review,reply,star,store
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요,이렇게 저희 매장 찾아주셔서 감사드립니다 맛 뿐만 아니라 양도 서비스도 모두 최상의...,5,
1,리뷰가 늦었네요 맛있게 잘 먹었습니다 또 시킬게요,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,5,
2,잘 먹었습니다 다만 주문에 있던 펩시콜라는 받지 못했습니다,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다 저희를 찾아 주신 만큼 1...,5,
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다 저희 매장의 메뉴를 제대로 즐기신 거 ...,5,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,5,


### 가게명 키워드 reply에서 삭제

In [13]:
# 가게명이 있는 idx 번호 확인
remove_name_idx = []
for name in short_name:
    idxs = train[train['reply'].str.contains(name)].index
    for idx in idxs:
        remove_name_idx.append(idx)
remove_name_idx = [sorted(set(remove_name_idx))][0]
len(remove_name_idx)

9731

In [14]:
# reply에 가게명이 있는 행만 별도 추출
train_remove = train.loc[remove_name_idx]
train_remove

,review,reply,star,store
16,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 된장찌개도 엄청 짜고...,저희 1988응답하라추억의옛날도시락의 메뉴를 주문해 주셔서 감사합니다 양이 줄었다고...,4,1988응답
23,20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다 배달이 늦는 점에 대한...,안녕하세요 1988응답하라추억의옛날도시락 입니다 남겨 주신 리뷰 잘 읽어보았습니다 ...,2,1988응답
29,양도 많고 진짜 맛있어요 시원한 묵사발로 더위도 가라앉히고 밥이랑 고기 양도 많아서...,저희 매장을 선택해주시고 리뷰 남겨주셔서 감사합니다 메뉴를 드시다가 양이 모자라면 ...,5,"플러스,플러"
43,맛이나 양은 무난해요 그런데 배달예정시간이 무색하게 늦게도착 차라리 배달 예정시간을...,1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다 저희 매장에서는 배달로 드...,3,1988응답
47,메인은 좋았는데 아 반찬이 다 쉬어 모두 버렸어요 한번 씹는데 촉감 맛이 이상해요 ...,안녕하세요 1988응답하라추억의옛날도시락 입니다 우선 감사하게도 저희 매장을 찾아 ...,3,1988응답
...,...,...,...,...
19388,저녁이라 그런가 봐 좀 덜 미끌미끌한 회를 맛볼 수 있었만 저녁이라 그런지 몰라도 ...,저희 윤희횟집을 방문해 주셔서 감사합니다 회가 조금 좋지 못했지만 앞으로는 더 좋은...,4,"윤희횟집,윤희횟,윤희"
19389,이 씨 부부의 경우 임신 8개월째로 배 속 아기가 근심되어 허리를 약간 굽히고 한 ...,저희 윤희횟집을 방문해 주신 여러분께 감사드립니다 앞으로도 더욱 좋은 서비스와 최선...,4,"윤희횟집,윤희횟,윤희"
19390,매운탕은 잘먹었고 회는 별로 안싱싱해서 매운탕은 못먹었어요,저희 윤희횟집을 방문해 주신 여러분께 감사의 말씀을 전하며 더 좋은 요리와 서비스로...,4,"윤희횟집,윤희횟,윤희"
19391,맛있게 잘 먹었습니다 감사합니다 맛있는 음식 많이 만들어 주세요,저희 윤희횟집을 방문해 주신 여러분께 감사드립니다 별 하나가 빠져서 아쉽기는하지만 ...,4,"윤희횟집,윤희횟,윤희"


In [15]:
# store 를 리스트로 변환(반복문을 위해)
train_remove['store'] = train_remove['store'].str.split(',')
train_remove['store']

16              [1988응답]
23              [1988응답]
29             [플러스, 플러]
43              [1988응답]
47              [1988응답]
              ...       
19388    [윤희횟집, 윤희횟, 윤희]
19389    [윤희횟집, 윤희횟, 윤희]
19390    [윤희횟집, 윤희횟, 윤희]
19391    [윤희횟집, 윤희횟, 윤희]
19392    [윤희횟집, 윤희횟, 윤희]
Name: store, Length: 9731, dtype: object

In [16]:
removed_reply = []
for reply, store_words in zip(train_remove['reply'], train_remove['store']):
    reply_words = reply.split()
    new_reply_words = []
    for reply_word in reply_words:
        if not any(store_word in reply_word for store_word in store_words): # store_words에 있는 단어가 어떤 단어에도 포함되지 않으면 추가
            new_reply_words.append(reply_word)

    new_reply = ' '.join(new_reply_words)
    removed_reply.append(new_reply)

train_remove['reply'] = removed_reply
train_remove['reply'][19392]

'저희 방문해 주신 여러분께 감사의 뜻을 전하고 싶고 앞으로도 고객만족을 위해 더욱 노력하겠습니다'

In [17]:
# train 셋에 반영
train.loc[train.index.isin(train_remove.index), 'reply'] = train_remove['reply']
train.iloc[19392]

review                           아주 맛있게 잘 먹었고 음식도 아주 정갈하네요 
reply     저희 방문해 주신 여러분께 감사의 뜻을 전하고 싶고 앞으로도 고객만족을 위해 더욱 ...
star                                                      4
store                                           윤희횟집,윤희횟,윤희
Name: 19392, dtype: object

In [18]:
train

,review,reply,star,store
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요,이렇게 저희 매장 찾아주셔서 감사드립니다 맛 뿐만 아니라 양도 서비스도 모두 최상의...,5,
1,리뷰가 늦었네요 맛있게 잘 먹었습니다 또 시킬게요,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,5,
2,잘 먹었습니다 다만 주문에 있던 펩시콜라는 받지 못했습니다,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다 저희를 찾아 주신 만큼 1...,5,
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다 저희 매장의 메뉴를 제대로 즐기신 거 ...,5,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,5,
...,...,...,...,...
19407,맛있게 잘 먹었습니다,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4,
19408,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4,
19409,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4,
19410,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4,


In [19]:
train['reply'][2554]

'제 가족이 먹는다는 생각으로 항상 청결하고 신선한 식재료로 맛있게 조리해드리고 있습니다 혹시라도 아쉬웠던 점이 있으시면 말씀해주시고 더 좋은 매장이 될 수 있게 최선을 다하겠습니다 고객님의 소중한 리뷰 정말 감사드리며 앞으로도 변하지 않는 한결같음으로 고객님들과 오래오래 함께하겠습니다 '

### 지점명 reply에서 삭제

In [20]:
for keyword in region_name:
    train['reply'] = train['reply'].str.replace(keyword, '')

- 개별 삭제

In [21]:
print(train[train['reply'].str.contains("헬리오시티")].index)

remove_keywords = ["당신은 지금 치킨이","헬리오시티",]
for keyword in remove_keywords:
    train['reply'] = train['reply'].str.replace(keyword, '')

train['reply'][414]

Int64Index([509, 18782, 18850, 19013], dtype='int64')


'떡볶이는 콜라죠 이렇게 저와 맘이 통하는 댓글을 보니 마음이 기쁘네요 앞으로도 비밀유지에 만전을 기하게습니다 '

- 여기까지 완료된 데이터 저장하기

In [104]:
# train.to_csv(f"{DATA_PATH}yogiyo_paraphrase.csv", index=False)

### 여기까지 완료된 데이터 불러오기

In [5]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_paraphrase.csv")

train['review'] = train['review'].astype(str)
train['reply'] = train['reply'].astype(str)
review, reply = train["review"] , train["reply"]
len(review) , len(reply)

(19412, 19412)

# 한국어 형태소 분석

In [6]:
review[:5]

0    사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요 
1                      리뷰가 늦었네요 맛있게 잘 먹었습니다 또 시킬게요 
2                 잘 먹었습니다 다만 주문에 있던 펩시콜라는 받지 못했습니다 
3                                굿 말이 필요 없는 메뉴 였습니다
4                                       맛있어요 양도많구좋음
Name: review, dtype: object

In [7]:
import unicodedata

def normalize(sentence):
    return unicodedata.normalize('NFC', sentence)

normalize(review[0])

'사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요 '

In [8]:
!pip install transformers

# 단어 사전 만들기

In [9]:
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2

class WordVocab():
    def __init__(self):
        self.word2index = {'<PAD>': PAD_TOKEN,
                           '<SOS>': SOS_TOKEN,
                           '<EOS>': EOS_TOKEN,}
        self.word2count = {}
        self.index2word = {PAD_TOKEN: '<PAD>',
                           SOS_TOKEN: '<SOS>',
                           EOS_TOKEN: '<EOS>'}
        self.n_words = 3  # PAD, SOS, EOS 포함

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
print(f'원문: {review[0]}')
lang = WordVocab()
lang.add_sentence(review[0])
print('단어사전: ', lang.word2index)

원문: 사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요 
단어사전:  {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '사진보다': 3, '실물이': 4, '백배': 5, '맛있어요': 6, '고기': 7, '양도': 8, '많아요': 9, '배달시간도': 10, '딱': 11, '맞춰서': 12, '보내주셨어요': 13, '': 14}


# 데이터셋 만들기

In [11]:
from konlpy.tag import Okt

In [12]:
class TextDataset(Dataset):
    def __init__(self, df, min_length=3, max_length=32):
        super().__init__()

        # TOKEN 정의
        self.PAD_TOKEN = 0 # Padding 토큰
        self.SOS_TOKEN = 1 # SOS 토큰
        self.EOS_TOKEN = 2 # EOS 토큰

        self.tagger = Okt()   # 형태소 분석기
        self.max_length = max_length # 한 문장의 최대 길이 지정

        # src: 리뷰, tgt: 답변
        src_clean = []
        tgt_clean = []
        store_list = []

        # 단어 사전 생성
        wordvocab = WordVocab()

        for _, row in df.iterrows():
            src = row['review']
            tgt = row['reply']

            # 한글 전처리
            src = self.clean_text(src)
            tgt = self.clean_text(tgt)

            if len(src.split()) > min_length and len(tgt.split()) > min_length:
                # 최소 길이를 넘어가는 문장의 단어만 추가
                wordvocab.add_sentence(src)
                wordvocab.add_sentence(tgt)
                src_clean.append(src)
                tgt_clean.append(tgt)

        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocab = wordvocab

    def clean_text(self, sentence):
        # 형태소 처리
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence

    def texts_to_sequences(self, sentence):
        # 문장 -> 시퀀스로 변환
        return [self.wordvocab.word2index[w] for w in sentence.split()]

    def pad_sequence(self, sentence_tokens):
        # 문장의 맨 끝 토큰은 제거
        sentence_tokens = sentence_tokens[:(self.max_length-1)]
        token_length = len(sentence_tokens)

        # 문장의 맨 끝부분에 <EOS> 토큰 추가
        sentence_tokens.append(self.EOS_TOKEN)

        for i in range(token_length, (self.max_length-1)):
            # 나머지 빈 곳에 <PAD> 토큰 추가
            sentence_tokens.append(self.PAD_TOKEN)
        return sentence_tokens

    def __getitem__(self, idx):
        inputs = self.srcs[idx]
        inputs_sequences = self.texts_to_sequences(inputs)
        inputs_padded = self.pad_sequence(inputs_sequences)

        outputs = self.tgts[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)

        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)

    def __len__(self):
        return len(self.srcs)

## 데이터셋 분할(테스트 데이터 생성)

In [13]:
MAX_LENGTH = 32
dataset = TextDataset(train, min_length=3, max_length=MAX_LENGTH)

In [31]:
# split_ratio = 0.2  # 테스트 데이터의 비율(20%)
# split_index = int(len(train) * split_ratio)

# test_data = train[:split_index] # 증강데이터가 없는 앞에서부터 20% 할당
# train_data = train[split_index:]

# len(test_data) , len(train_data)

(3882, 15530)

In [32]:
# train_dataset = TextDataset(train_data, min_length=3, max_length=MAX_LENGTH)
# test_dataset = TextDataset(test_data, min_length=3, max_length=MAX_LENGTH)

In [14]:
x, y = dataset[0]

In [15]:
print(f'x shape: {x.shape}')
print(x)

x shape: torch.Size([32])
tensor([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,  2,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [16]:
print(f'y shape: {y.shape}')
print(y)

y shape: torch.Size([32])
tensor([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 10, 28, 14, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40,  6, 41, 42, 43, 31, 44,  2])


In [17]:
from torch.utils.data import random_split

train_size = int(len(dataset) * 0.8) # 80%의 데이터를 train에 할당
test_size = len(dataset) - train_size # 나머지 20% 데이터를 test에 할당
print(train_size, test_size)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size]) # 랜덤 스플릿으로 분할

12040 3010


# 데이터로더 만들기

In [18]:
from torch.utils.data import DataLoader, SubsetRandomSampler

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [19]:
# 1개의 배치 데이터를 추출
x, y = next(iter(train_loader))

# shape: (batch_size, sequence_length)
x.shape, y.shape

(torch.Size([16, 32]), torch.Size([16, 32]))

# Encoder 만들기

In [20]:
class Encoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        super().__init__()

        # 단어 사전의 개수 지정
        self.num_vocabs = num_vocabs
        # 임베딩 레이어 정의 (number of vocabs, embedding dimension)
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        # GRU (embedding dimension)
        self.gru = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)
        output, hidden = self.gru(x)
        return output, hidden

In [21]:
embedding_dim = 64 # 임베딩 차원
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

# x의 shape을 변경합니다.
# (batch_size, sequence_length) => (sequence_length, batch_size)
embedded = embedding(x)

print(x.shape)
print(embedded.shape)
# input:  (sequence_length, batch_size)
# output: (sequence_length, batch_size, embedding_dim)

torch.Size([16, 32])
torch.Size([16, 32, 64])


In [22]:
embedded = embedded.permute(1, 0, 2)
print(embedded.shape)
# (sequence_length, batch_size, embedding_dim)

torch.Size([32, 16, 64])


In [23]:
hidden_size = 32

gru = nn.GRU(embedding_dim,      # embedding 차원
             hidden_size,
             num_layers=1,
             bidirectional=False)

# input       : (sequence_length, batch_size, embedding_dim)
# h0          : (Bidirectional(1) x number of layers(1), batch_size, hidden_size)
o, h = gru(embedded, None)

print(o.shape)
print(h.shape)
# output      : (sequence_length, batch_size, hidden_size x bidirectional(1))
# hidden_state: (bidirectional(1) x number of layers(1), batch_size, hidden_size)

torch.Size([32, 16, 32])
torch.Size([1, 16, 32])


In [24]:
NUM_VOCABS = dataset.wordvocab.n_words
print(f'number of vocabs: {NUM_VOCABS}')

number of vocabs: 25110


## Encoder 정의

In [25]:
# Encoder 정의
encoder = Encoder(NUM_VOCABS, hidden_size=32, embedding_dim=64, num_layers=1)

In [26]:
# Encoder에 x 통과 후 output, hidden_size 의 shape 확인
# input(x)    : (batch_size, sequence_length)
o, h = encoder(x)

print(o.shape)
print(h.shape)
# output      : (sequence_length, batch_size, hidden_size x bidirectional(1))
# hidden_state: (bidirectional(1) x number of layers(1), batch_size, hidden_size)

torch.Size([32, 16, 32])
torch.Size([1, 16, 32])


# Decoder 만들기

In [27]:
class Decoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers=1, dropout=0.2):
        super().__init__()
        # 단어사전 개수
        self.num_vocabs = num_vocabs
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

        # 최종 출력은 단어사전의 개수
        self.fc = nn.Linear(hidden_size, num_vocabs)

    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) # (1, batch_size) 로 변환
        embedded = F.relu(self.embedding(x))
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden_state)
        output = self.fc(output.squeeze(0)) # (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
        return output, hidden

In [28]:
x = torch.abs(torch.randn(size=(1, 16)).long())
print(x)
x.shape
# batch_size = 16 이라 가정했을 때,
# (1, batch_size)
# 여기서 batch_size => (1, batch_size) 로 shape 변환을 선행

tensor([[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0]])


torch.Size([1, 16])

In [29]:
embedding_dim = 64 # 임베딩 차원
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

embedded = embedding(x)
embedded.shape
# embedding 출력
# (1, batch_size, embedding_dim)

torch.Size([1, 16, 64])

In [30]:
hidden_size = 32

gru = nn.GRU(embedding_dim,
             hidden_size,
             num_layers=1,
             bidirectional=False,
             batch_first=False, # batch_first=False로 지정
            )

o, h = gru(embedded)

print(o.shape)
# output shape: (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
print(h.shape)
# hidden_state shape: (Bidirectional(1) x number of layers(1), batch_size, hidden_size(32))

torch.Size([1, 16, 32])
torch.Size([1, 16, 32])


In [31]:
fc = nn.Linear(32, NUM_VOCABS) # 출력은 단어사전의 개수로 가정

output = fc(o[0])

print(o[0].shape)
print(output.shape)
# input : (batch_size, output from GRU)
# output: (batch_size, output dimension)

torch.Size([16, 32])
torch.Size([16, 25110])


## Decoder 정의

In [32]:
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words, hidden_size=32, embedding_dim=64, num_layers=1)

In [33]:
x, y = next(iter(train_loader))

o, h = encoder(x)

print(o.shape, h.shape)
# output      : (batch_size, sequence_length, hidden_size(32) x bidirectional(1))
# hidden_state: (Bidirectional(1) x number of layers(1), batch_size, hidden_size(32))

torch.Size([32, 16, 32]) torch.Size([1, 16, 32])


In [34]:
x = torch.abs(torch.full(size=(16,), fill_value=SOS_TOKEN).long())
print(x)
x.shape
# batch_size = 16 이라 가정(16개의 SOS 토큰)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


torch.Size([16])

In [35]:
decoder_output, decoder_hidden = decoder(x, h)
decoder_output.shape, decoder_hidden.shape
# (batch_size, num_vocabs), (1, batch_size, hidden_size)

(torch.Size([16, 25110]), torch.Size([1, 16, 32]))

# Seq2Seq 모델 만들기

In [36]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        # inputs : (batch_size, sequence_length)
        # outputs: (batch_size, sequence_length)

        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_vocabs

        # 리턴할 예측된 outputs를 저장할 임시 변수
        # (sequence_length, batch_size, num_vocabs)
        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)

        # 인코더에 입력 데이터 주입, encoder_output은 버리고 hidden_state 만 살립니다.
        # 여기서 hidden_state가 디코더에 주입할 context vector 입니다.
        # (Bidirectional(1) x number of layers(1), batch_size, hidden_size)
        _, decoder_hidden = self.encoder(inputs)

        # (batch_size) shape의 SOS TOKEN으로 채워진 디코더 입력 생성
        decoder_input = torch.full((batch_size,), SOS_TOKEN, device=self.device)

        # 순회하면서 출력 단어를 생성합니다.
        # 0번째는 SOS TOKEN이 위치하므로, 1번째 인덱스부터 순회합니다.
        for t in range(0, output_length):
            # decoder_input : 디코더 입력 (batch_size) 형태의 SOS TOKEN로 채워진 입력
            # decoder_output: (batch_size, num_vocabs)
            # decoder_hidden: (Bidirectional(1) x number of layers(1), batch_size, hidden_size), context vector와 동일 shape
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)

            # t번째 단어에 디코더의 output 저장
            predicted_outputs[t] = decoder_output

            # teacher forcing 적용 여부 확률로 결정
            # teacher forcing 이란: 정답치를 다음 RNN Cell의 입력으로 넣어주는 경우. 수렴속도가 빠를 수 있으나, 불안정할 수 있음
            teacher_force = random.random() < teacher_forcing_ratio

            # top1 단어 토큰 예측
            top1 = decoder_output.argmax(1)

            # teacher forcing 인 경우 ground truth 값을, 그렇지 않은 경우, 예측 값을 다음 input으로 지정
            decoder_input = outputs[:, t] if teacher_force else top1

        return predicted_outputs.permute(1, 0, 2) # (batch_size, sequence_length, num_vocabs)로 변경

In [37]:
# Encoder 정의
encoder = Encoder(num_vocabs=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Seq2Seq 정의
seq2seq = Seq2Seq(encoder, decoder, 'cpu')

In [38]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)
# (batch_size, sequence_length), (batch_size, sequence_length)

torch.Size([16, 32]) torch.Size([16, 32])


In [39]:
output = seq2seq(x, y)
print(output.shape)
# (batch_size, sequence_length, num_vocabs)

torch.Size([16, 32, 25110])


## 모델 확인

In [40]:
NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_SIZE = 512
EMBEDDIMG_DIM = 256

print(f'num_vocabs: {NUM_VOCABS}\n======================')

# Encoder 정의
encoder = Encoder(num_vocabs=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)

# Seq2Seq 생성
# encoder, decoder를 device 모두 지정
model = Seq2Seq(encoder.to(device), decoder.to(device), device)
print(model)

num_vocabs: 25110
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(25110, 256)
    (gru): GRU(256, 512)
  )
  (decoder): Decoder(
    (embedding): Embedding(25110, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (gru): GRU(256, 512)
    (fc): Linear(in_features=512, out_features=25110, bias=True)
  )
)


# 하이퍼파라미터 정의

In [41]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0

        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta


    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')

        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')


        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [42]:
LR = 1e-3
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
es = EarlyStopping(patience=5, delta=0.001, mode='min', verbose=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

# 학습 loop 만들기

In [43]:
def train(model, data_loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0

    for x, y in data_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        # output: (batch_size, sequence_length, num_vocabs)
        output = model(x, y)
        output_dim = output.size(2)

        # 1번 index 부터 슬라이싱한 이유는 0번 index가 SOS TOKEN 이기 때문
        # (batch_size*sequence_length, num_vocabs) 로 변경
        output = output.reshape(-1, output_dim)

        # (batch_size*sequence_length) 로 변경
        y = y.view(-1)

        # Loss 계산
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

    return running_loss / len(data_loader)

In [44]:
def evaluate(model, data_loader, loss_fn, device):
    model.eval()

    eval_loss = 0

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)

            # Loss 계산
            loss = loss_fn(output, y)

            eval_loss += loss.item() * x.size(0)

    return eval_loss / len(data_loader)

# 추론 함수

In [45]:
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS_TOKEN, EOS_TOKEN, PAD_TOKEN]:
            outputs.append(word)
        if word == EOS_TOKEN:
            break
    return ' '.join(outputs)

In [46]:
!git clone https://github.com/ssut/py-hanspell.git
!cd py-hanspell
!pip install py-hanspell/

fatal: destination path 'py-hanspell' already exists and is not an empty directory.
Processing ./py-hanspell
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=955e425207e43cd12583609ed9e6cc813ee679fa38ce100aa8a5f54a58dcea0e
  Stored in directory: /root/.cache/pip/wheels/a3/a7/16/8db596361f402638188412108152b8fbf6db76c898e00fe02d
Successfully built py-hanspell
  Attempting uninstall: py-hanspell
    Found existing installation: py-hanspell 1.1
    Uninstalling py-hanspell-1.1:
      Successfully uninstalled py-hanspell-1.1


In [47]:
from hanspell import spell_checker
def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

# 테스트 loop 만들기
- 챗봇 함수화

In [48]:
# 전체 예측값을 데이터프레임 형식에 담기
def random_evaluation(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            data = {'test_review': [], 'chatbot_reply': []}
            for i in range(n_samples):
                x_result = sequence_to_sentence(x[i], index2word)
                y_result = sequence_to_sentence(y[i], index2word)
                pred_result = sequence_to_sentence(preds[i].argmax(1), index2word)
                pred_result_correct = correct_spell(pred_result)
                data['test_review'].append(x_result)
                data['chatbot_reply'].append(pred_result_correct)

    df = pd.DataFrame(data)
    print(df)

In [49]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(text, generated_text):
    y = [text]  # 실제 데이터 y
    X = generated_text  # 생성된 텍스트 X

    # BLEU 점수 계산
    bleu_score = sentence_bleu(y, X)
    return bleu_score

In [50]:
# 10개만 보기
def random_evaluation_10(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            for i in range(n):
                result_review = sequence_to_sentence(x[i], index2word)
                result_reply = sequence_to_sentence(y[i], index2word)
                result_pred = sequence_to_sentence(preds[i].argmax(1), index2word)
                print(f'리뷰    : {result_review}')
                print(f'답변    : {result_reply}')
                print(f'예측답변: {correct_spell(result_pred)}')
                print(f'BLEU점수: {calculate_bleu(result_reply, result_pred)}')
                print('==='*10)

# 학습하기

In [51]:
# yogiyo_paraphrase
NUM_EPOCHS = 20
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase.pt'

best_loss = np.inf

for epoch in tqdm(range(NUM_EPOCHS), total=NUM_EPOCHS):
    loss = train(model, train_loader, optimizer, loss_fn, device)

    val_loss = evaluate(model, test_loader, loss_fn, device)

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_PATH)

    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')

    # Early Stop
    es(loss)
    if es.early_stop:
        break

    # Scheduler
    scheduler.step(val_loss)

model.load_state_dict(torch.load(STATEDICT_PATH))
torch.save(model.state_dict(), f'/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase{best_loss:.4f}.pt')

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 1, loss: 79.5517, val_loss: 64.7468
[EarlyStopping] (Update) Best Score: 79.55168
[EarlyStopping] (Update) Best Score: 57.83777
[EarlyStopping] (Update) Best Score: 51.04923
[EarlyStopping] (Update) Best Score: 46.54832
[EarlyStopping] (Update) Best Score: 43.59740
epoch: 6, loss: 39.5366, val_loss: 51.5719
[EarlyStopping] (Update) Best Score: 39.53658
[EarlyStopping] (Update) Best Score: 37.41048
[EarlyStopping] (Update) Best Score: 34.78471
Epoch 00008: reducing learning rate of group 0 to 5.0000e-04.
[EarlyStopping] (Update) Best Score: 30.78729
[EarlyStopping] (Update) Best Score: 28.13335
epoch: 11, loss: 26.7883, val_loss: 53.7014
[EarlyStopping] (Update) Best Score: 26.78833
Epoch 00011: reducing learning rate of group 0 to 2.5000e-04.
[EarlyStopping] (Update) Best Score: 24.41597
[EarlyStopping] (Update) Best Score: 23.49104
[EarlyStopping] (Update) Best Score: 22.50856
Epoch 00014: reducing learning rate of group 0 to 1.2500e-04.
[EarlyStopping] (Update) Best Score: 21.

# 결과 확인하기

In [67]:
# yogiyo 전체데이터 모델링
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 먹을만 합니다 추천 해 요
답변    : 좋은 리뷰 감사 드립니다 저희 은 리뷰 평점 이나 표면 적 으로만 좋은 집 이 아닌 드셧 던 분들 이 믿고 다시 드시고싶은 집 이 되자 라는 마음 으로 모두 가
예측답변: 고객 님 소중한 고객 네 이 이임 이임 맛있게 드셨다니 너 어어 어 무 기쁜 네 용 만족스러운 식사하셨다면에 용 만족스러운 식사하셨다면 찜 가
BLEU점수: 1.7456357302038282e-78
리뷰    : 여친 이 너무 맛있게 잘 먹었답니다 감사합니다 사장 님 번창 하세요
답변    : 안녕하세요 토핑 을 아끼지 않는 미 미 주문 주시 고 리뷰 로 찾아와 주셔서 정말 감사 드립니다 앞 으로도 맛있는 생각나실 때 저희 미 또 찾아주세요 감사합니다
예측답변: 맛있게 드셔주셔서 감사합니다 요청 사항 은 조 리전 늘 꼼꼼하게 체크하고 있으니 필요하신 사항 있으시면 다음 주 문 시 주 문 메모에 남겨주세요 최대한 반영해서 보내 드리겠습니다 믿고 주문해
BLEU점수: 0.1556563403952443
리뷰    : 주말 의 마무리 는 역시
답변    : 고객 님 리뷰 에 몸 이 절로 들썩입니다 시간 날 때 마다 리뷰 훑어 보며 반응 살피 고 있는데 덕분 에 넘 힘 이나고 힐링 이 되네요 리뷰 너무 감사 드립니다
예측답변: 안녕하세요 고객 님 소중한 리뷰 남겨주셔서 감사합니다 저희 큰아들 백 순대는 항상 맛있는 서비스를 해드리기 위해 서 노력하고 있습니다 사용되는 모든 곱창 은 국내 산 제 주산
BLEU점수: 0.12288378213016357
리뷰    : 고기 가 맛있어요 사장 님 이 서비스 로 쌈장 도 주셨어요 배달 도 예상 시간 보다 빠흐 게 옵니다
답변    : 저희 목살 만족스럽고 맛있게 잘 드셨다니 기쁩니다 소중한 리뷰 와 저희 에게 큰 힘 을 주는 좋은 평가 남겨 주셔서 정말 감사 드립니다 무더운 날씨 에 건강 잘 지키시고 오늘 도
예측답변: 맛있게 드셔 주시 고 소중한 리뷰 남겨주셔서 감사합니다 만족스러운 식사하신 것 같아 오

In [ ]:
# yogiyo 전체데이터 모델링
STATEDICT_OV_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase_clean.pt'
model.load_state_dict(torch.load(STATEDICT_OV_PATH))
random_evaluation_10(model, test_dataset, train_dataset.wordvocab.index2word, device)

리뷰    : 주문 에 후라이드 잇 슈가 있었지만 다 맛있었습니다
답변    : 저희 을 항상 찾아주시고 성원 해주시는 분 들 께 감사 의 뜻 을 전하 고 싶을 만큼 귀한 시간 내어 주셔서 감사합니다 .
예측답변: 항상 저희 직구 삼이 직구 삼이 되도록 노력 직구 삼이 되도록 노력 하겠습니다 감사합니다 .
BLEU점수: 0.14828218247800706
리뷰    : 맛있습니다 배달 도 빠르고 양도 많아오
답변    : 항상 건강하고 행복한 나날 들 이 되시길 기원 합니다 . 고객 님 들의 소중한 정보 감사 드립니다 .
예측답변: 항상 건강하고 행복한 나날 들 이 되시길 기원 합니다 .
BLEU점수: 0.4464393987758162
리뷰    : 디저트 하나 하나 다 맛있어요 휘 낭시 에 랑 소금 빵 완전 맛있어요 이 즈 니 버터 프레첼 은 팥 앙금 들어있는건 줄 알았는데 다시 보니 아니더라 규 요 그치만 이
답변    : 이 즈 니 버터 프리첼 의 팥 앙금 버전 앙 버터 도 뚜레쥬르 팀 에 제안 해 보겠습니다 .
예측답변: 이 즈 니 버터 프리첼 의 팥 앙금 버전 앙 버터 도 뚜레쥬르 팀 에 제안 해 보겠습니다 .
BLEU점수: 1.0
리뷰    : 국물 이 깔끔하고 맛있어요 또 먹고 싶은 맛 이네 요
답변    : 저희 국밥 을 많이 사랑 해 주세요 :) 항상 최선 을 다해 준비 하는 만큼 더 좋은 맛 과 영양 으로 보답 하겠습니다 .
예측답변: 저희 국밥 을 많이 사랑 해 주세요 :) 항상 최선 을 다해 준비 하는 만큼 더 좋은 맛 과 영양 으로 보답 하겠습니다 .
BLEU점수: 1.0
리뷰    : 커피 도 부드럽고 다 맛 나네요 잘 먹었습니다
답변    : 항상 정갈한 브런치 맛있는 브런치 준비 하고 고객 님 기다릴게요 곧 또 만나요 그때 까지 행복한 시간 되세요 .
예측답변: 항상 정갈한 브런치 는 브런치 준비 가 준비 기다릴게요 기다릴게요 곧 또 찾아주세요 행복한 하루 되세요 .
BLEU점수: 0.5833316665807692
리뷰    : 맛

In [ ]:
# # yogiyo 전체데이터 모델링
# STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_clean.pt'
# model.load_state_dict(torch.load(STATEDICT_PATH))
# random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 진한 미역국 한 그릇 맛있게 먹었습니다
답변   : 안녕하세요 맛 나게 드셨다니 정말 감사합니다 뚝배기 한 그릇 씩 조리 하고 있어서 재료 양 맛 의 풍 미 식감 등등 을 항상 일정하게 조리 할수
예측답변: 감사합니다 좋은 휴일 되십시요
질문   : 맛 나게 잘먹엇슴돠 감사함 돠
답변   : 저희 목살 맛있게 잘 드셨다니 기쁘고 힘 이 납니다 소중한 리뷰 와 저희 에게 큰 힘 이 되는 좋은 평가 도 남겨 주셔서 정말 감사 드립니다
예측답변: 고객 님 주문 해 주셔서 감사합니다 맛있게 드시고 예쁜 리뷰 남겨주셔서 감사해요 피스타치오 가 갈리는 품목 이 프루타 과일 과 디저트 로 비타민 충전
질문   : 포장 이 너무 예뻐요 진짜 가서 먹는듯 한 플레이 팅 파스타 에 마늘 이 듬뿍 들어있더라고요 역시 마늘 의 민족 미트볼 이랑 가라 아케 도 맛있었어요
답변   : 명작 파스타 입니다 맛있게 드셨다니 저희 도 기분 이 좋습니다 정성스런 리뷰 도 너무 감사 드립니다 앞 으로도 더 맛있는 파스타 제공 드릴 수 있도록 직원
예측답변: 감사합니다 좋은 밤 되세요
질문   : 어렸을 때 부터 시켜 먹던 오래된 맛집 이 예요 맛있고 배달 도 빠르고 좋습니다
답변   : 안녕하세요 좋은 기억 의 만다린 이네요 감사 드립니다 계속 섬심 성의 를 다해 모시겠습니다 건강하세요
예측답변: 안녕하세요 마라하 오죽 전 점 입니다 주문 해주시고 잘 드셔주셔서 정말 감사해요 덕분 에 행복지수 마구 상승 하네요 저희 음식 드시면서 힘 많이 내셨음 좋겠습니다 즐거운
질문   : 분식점 으로 충실한 그 맛 식후감
답변   : 짧지만 강렬한 리뷰 감사 드립니다 깨 돌이 김밥 의 떡볶이 소스 는 캡사이신 이 들어가지 않습니다 고추장 과 청양고추 가루 로 매운 맛 을 내서 추억
예측답변: 감사합니다 좋은 밤 되세요
질문   : 국밥 집 중 에 젤 맛있어요 굿
답변   : 고객 님 소중한 고객 니 이 이임 국밥 집 중 에 젤 맛있다니 지금 까지 본 리뷰 중 에 제일 뿌듯하고 힘 이 나 는 말

In [ ]:
# # yogiyo 오버샘플 모델링(18000개 데이터)
# STATEDICT_OV_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_OV_clean.pt'
# model.load_state_dict(torch.load(STATEDICT_OV_PATH))
# random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 이 걸 뭐라고 해야 할지 아직도 안 왔네요 배달 예 정시 간 이 30분 인데 65분 지나도 안 옴
답변   : 배송 이 늦어진 부분 정말 죄송합니다 조리 는 빨리 되나 저희 가 이용 하고 있는 배달 대행 업체 의 문제 로 피크타임 배송 이 늦는 경우
예측답변: 배송 이 늦어진 부분 정말 죄송합니다 조리 는 빨리 되나 저희 가 이용 하고 있는 배달 대행 업체 의 문제 로 피크타임 배송 이 늦는 경우
질문   : 음식 의 양은 많지만 맛 이 있지는 않네요 다 못 먹고 남겼습니다 사진 에 있는게 다 남은 음식 이에요 특히 절 인 양파 는 무슨 맛
답변   : 안녕하세요 고객 님 정말 정말 죄송합니다 부족한 부분 으로 실망 시켜 드린 점 사과 드리며 항상 저희 는 리뷰 덕분 에 더욱 책임감 을 갖게 되는데요
예측답변: 안녕하세요 고객 님 정말 정말 죄송합니다 부족한 부분 으로 실망 시켜 드린 점 사과 드리며 항상 저희 는 리뷰 덕분 에 더욱 책임감 을 갖게 되는데요
질문   : 이렇게 왔네요 기분 나빠서 버렸어요
답변   : 왕 빈자 삼파전 입니다 고객 님 믿고 주문 해주셨는데 불편 을 드려서 죄송합니다 항상 기본 적 으로 제일 신경 쓰고 있는 부분 인데 도 불구 하고
예측답변: 왕 빈자 삼파전 입니다 고객 님 믿고 주문 해주셨는데 불편 을 드려서 죄송합니다 항상 기본 적 으로 제일 신경 쓰고 있는 부분 인데 도 불구 하고
질문   : 오 돌뼈 순 한 맛 이라 전혀 맵 지 않았구요 계란찜 추가 했는데 맛있게 잘 먹었어요 배달 빨리 해주셨고 친절하셨습니다
답변   : 소중한 리뷰 감사합니다 순 한 맛 부탁 하셔서 순 한 맛 해드려는데 더 노력 하겠 습니다
예측답변: 양 이 맛 나게 에 맛있게 드셨다니 다행 새해 복 많이 애용 해주세요
질문   : 맛 은 평범합니다 영수증 에 주소 까지나 와있는데 전화하셔서 제대로 안 나와있다고 하셔서 사과 하고 음식 받았습니다 기분 나쁜 식사 만 아니면 더 좋을 뻔했네요
답변   : 기분 나쁘셨다니 죄송합니다 고객 님 이 

In [ ]:
# # yogiyo 오버샘플 모델링(중복 제거, 2만개 데이터)
# STATEDICT_OV_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_OV_clean.pt'
# model.load_state_dict(torch.load(STATEDICT_OV_PATH))
# random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 먹어 보고 부터 여기 가게 완전 반해서 여기 서 만 마라 탕 먹어요 두 명 이 마라 탕 취향 이 좀 달라 서 늘 둘다 맞는 가게 찾기 가 힘들었는데 두
답변   : 우아 50 번 저희 는 그럼 고객 님 50 번 사랑 한다고 말 하고 싶네요 너무 너무 감사합니다 어깨 가 으쓱 으쓱 더 열심히 노력 하겠습니다
예측답변: 안녕하세요 당신 은 지금 치킨 이 당긴다 헬리 오 시티 점 입니다 700 아닌데 는 고객 님 에서 10분 분 분 분 좋으시겠어요 밀려오고 로 이 많아요 보니 트리플 이 조금 이나마
질문   : 잘 먹었습니다 지난 리뷰 에 사장 님 이 직접 댓글 도 해주셨네요 좋은 음식 으로 대박 나시기 바랍니다
답변   : 조영훈 의 족발 보쌈 을 믿고 주문 해 주셔서 감사합니다 도시락 세트 는 혼자 서 드시 기 엔 넉넉한 양 이 나가 든든하게 드실 수 있는 양 입니다 모든 음식 은
예측답변: 안녕하세요 고객 님 별 점 리뷰 진심 으로 감사합니다 부족한 부분 이 있었는데도 이렇게 극찬 과 함께 별 점 만점 까지 주시 고 진심 으로 감사 드립니다 항상 맛있는 음식 과
질문   : 그냥 그저 그럼 그냥 그저 그럼
답변   : 안녕하세요 고객 님 고객 님 입맛 엔 안 맞으셨군요 불편 사항 요청 사항 있으시면 언제 든지 연락 주시 면 고객 님 입맛 에 맞게 최대한 노력 하겠습니다 감사합니다
예측답변: 안녕하세요 고객 님 고객 님 입맛 엔 안 맞으셨군요 불편 사항 요청 사항 있으시면 언제 든지 연락 주시 면 고객 님 입맛 에 맞게 최대한 노력 하겠습니다 감사합니다
질문   : 배달 은 50분 예정 이었는데 15분 늦었구요 치즈스틱 은 드시지 마세요
답변   : 주문 감사합니다 피자 는 빨리 들렀으나 대행 분 이 너무 늦게 전 해드리는 것 같더라고요 아쉬운 점 보완 해 보도록 할게요 죄송합니다
예측답변: 주문 감사합니다 피자 는 빨리 들렀으나 대행 분 이 너무 늦게 전 해드리는 것 같더라고요 아쉬운 점 보완 해 보도록 할게요 죄송합니다
질문   : 진짜 맛있어요

In [ ]:
# # yogiyo 오버샘플 모델링(중복 제거, 가게명 제거, 2만개 데이터)
# STATEDICT_OV_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_OV_clean_store_del.pt'
# model.load_state_dict(torch.load(STATEDICT_OV_PATH))
# random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 자주 먹지만 항상 맛있네요
답변   : 고객 님 안녕하세요 골라 먹는 장닭 치킨 부천대 점 입니다 이번 주문 에 맛있게 드셔 주셔서 정말 감사합니다 맛있게 드셨다는 말씀 은 언제 들어도 기분 이 좋아지고 또 이렇게 믿고 찾아 주시 는 고객 님 들을 실망 시켜 드리지 않도록 더 열심히 해야겠다는 생각 이 들게 하는
예측답변: 고객 님 소중한 주문과 리뷰 감사드립니다 드립니다 주문해 주신 대로 만인 감사드려요 다음에 또 주문해 주세요
질문   : 딱 원하던 맛 이었어요 양도 푸 짐 하고 잘 먹었습니다
답변   : 안녕하세요 만족스러운 식사 가 되신 것 같아 다행 입니다 예쁜 사진 과 힘 이 나 는 행복 의 리뷰 진심 으로 감사합니다 다음 에는 더 신경 써서 배달 해 드리겠습니다 항상 건강 잘 챙기시고 행복한 웃는 날 만 가득하세요
예측답변: 이쁜 리뷰 감사합니다 고객 님께서 알촌 을 찾아주셔서 너무 기쁩니다 고객 님의 소중한 시간에 저희 알촌 이 되도록 하겠습니다 좋은 알촌 알촌 충북대 점 많이 많이 많이 주셔서 감사드려요 오늘 도 알촌 을 찾아주셔서 감사합니다 오늘 도 좋은 하루 보내세요
질문   : 양 진짜 넉넉하고 맛있네요 고기 도 부드럽구요 리뷰 이벤트 로 주신 쥬스 도 맛있어요
답변   : 안녕하세요 고기 는 수비드 하여 더욱 부드럽게 조리 하고 있습니다 맛 과 양 영양 까지 생각 하여 든든한 한 끼 전달 드리겠 습 니 대 감사합니다
예측답변: 저희 양 갈비 맛있게 잘 드시고 이렇게 힘나는 소중한 리뷰 와 저희에게 큰 힘 이 되는 좋은 평 가까지 남겨 주셔서 정말 감사드립니다 더워진 날씨에 건강 잘 지키시고 알차고 즐거운 하루 보내세요 감사합니다
질문   : 그냥 여기는 믿고 드세요
답변   : 저희 청년 치킨 을 이용 해 주셔서 감사 드립니다 맛있는 치킨 과 푸 짐 한 양 신속한 서비스 로 보답 드릴게요 언제나 편히 이용 할 수 있도록 최선 을 다 하는 청년 치킨 이 되겠습니다 서비스 이용 시 서비스 가 불편한 점 이 있으시면 가게 로 전화

In [ ]:
# # yogiyo 오버샘플 모델링(중복 제거, 가게명 제거, subword tokenizer, 2만개 데이터)
# STATEDICT_OV_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_OV_clean_store_del_sub.pt'
# model.load_state_dict(torch.load(STATEDICT_OV_PATH))
# random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 맛 있 게 먹 었 내 요 헌 데 덮 밥 의 명 란 은
답변   : 혹 시 명 란 마 요 네 즈 가 안 들어 있 었 나 요 으 악 정 말 정 말 죄 송 합 니다
예측답변: 혹 시 명 란 마 요 네 즈 가 안 들어 있었 나 요 으 악 정 말 정 말 죄 송 합니다
질문   : 김 밥 엄 청 커 요
답변   : 맛 있 게 드 셨 는 지 요 항 상 푸 짐 하고 맛 있는 맛 으로 [unk] 자 주 애 용 해 주 세 요
예측답변: 안녕하세요 고 객 님 d 번 거로 우 실 텐 데 리 뷰 작 성해 주 셔 서 감사합니다 어 휴 도 가 금 계 가 되 면 괜 찮 아 서 된다 면
질문   : 아 이 건 좀 아 니 지 않 나 요 돈 까 스 가 이 렇게 안 익 어 서 오 는 건 처음 입 니다
답변   : 고 객 님 매 장으로 전 화 주 셨 으면 전 화 로 설 명 드 렸 을 텐 데 죄 송 합 니다 안 심 카 츠 같은 경우 미 오 글 로 빈 이 올 라 와 서 [unk] [unk] 혹 시 나 하는 경우
예측답변: 고 객 님 매 장으로 전 화 주 셨 으면 전 화로 설 명 드 였을 텐 데 죄 송 합니다 안 심 카 츠 같은 경우 미 오 글로 빈 이 올 라 와 서 [unk] [unk] 혹 시 나 하는 경우
질문   : 스 파 게 티 는 진 짜 맛 있 었 어 요 피 자는 너 무 [unk] 소 금 들이 부 으 신 듯
답변   : 소 스 가 많이 들어 간 것 같 네 요 싱 겁 다는 분 이 계 셔 좀 더 넣 으면 이 렇게 짜 지는 가 봐 요 담 엔 소 스 적 게 라고 메 모 주 시 면 [unk] 죄 송 합 니다 소 금
예측답변: 소 스 가 많이 들어간 것 같 네 요 싱 겁 다는 분 이 계 셔 좀 더 넣 으면 이렇게 짜지는가 봐 요 담 엔 소 스 적 게라고 메 모 주 시 면 [unk] 죄 송 합니다 소 금
질문   : 다 이어 트 하는 데 먹 기 좋 아 요
답변   : 다 이어 트 하 실 때 저 희 음 식 이 정 말 알 맞 죠 건 강 하고 맛 있는 다 이어 트 하 실 때 저 희 비 오 키 친 찾